In [1]:
from examinebugs import examine_rev

distmodels = ["BasicDistanceModel", "GitDiffDistModel", 
              "MossDistModel", "PatientDiffDistModel"]
scoremodels = ["SimpleScoreModel", "LongestScoreModel", 
               "ShortestScoreModel", "InEdgesScoreModel",
               "OutEdgesScoreModel", "SizeScoreModel", 
               "TimeWeightedScoreModel"]

Using MOSS_PATH=/home/cat/rzou/mosslocal/
USING MOSS_PATH=/mnt/ramdisk/mosstmp/


In [2]:
buggy_dict = examine_rev(
    '/home/cat/rzou/linux', 'mm', 'mm', 
    distmodels[3], scoremodels[4],
    '19be0eaffa3ac7d8eb6784ad9bdbc7d67ed8e619'
)

Read patch models


In [23]:
for line in (eval(buggy_dict[674]['patch']['content'])):
    print line

static struct page *no_page_table(struct vm_area_struct *vma,
		unsigned int flags)
{
	/*
	 * When core dumping an enormous anonymous area that nobody
	 * has touched so far, we don't want to allocate unnecessary pages or
	 * page tables.  Return error instead of NULL to skip handle_mm_fault,
	 * then get_dump_page() will return NULL to leave a hole in the dump.
	 * But we can only make this optimization where a hole would surely
	 * be zero-filled if handle_mm_fault() actually did handle it.
	 */
	if ((flags & FOLL_DUMP) && (!vma->vm_ops || !vma->vm_ops->fault))
		return ERR_PTR(-EFAULT);
	return NULL;
}

static struct page *follow_page_pte(struct vm_area_struct *vma,
		unsigned long address, pmd_t *pmd, unsigned int flags)


In [14]:
buggy_dict.keys()


[674, 2984, 1994, 14, 36534, 17944, 36314]

In [25]:
for pid, p in buggy_dict.iteritems():
    print "%d %d" % (pid, p['score'])

674 323
2984 369
1994 323
14 0
36534 66541
17944 35675
36314 66637
